In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

In [2]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh6FAh7FqEKRxP4yAx2aaaJLf0Tgcy2ZoxO4XgrHgDthNfnE80rVcLk

Successfully saved authorization token.


In [3]:
# Define geometry for data collection

roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);  # Nanda Bet S1 extent
roi_small = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet small region

# boundary shapefiles
shp_file = 'data/s1_boundary.shp'
s1_boundary = geemap.shp_to_ee(shp_file)
# type(s1_boundary)

# this is the main boundary to use for the small region around nanda bet
shp_file = 'data/smaller_region.shp'
small_boundary = geemap.shp_to_ee(shp_file)

# # this is the boundary to use just for the region north and east of the nanda bet island
# shp_file = 'data/nanda_bet_poly1.shp'
# small_boundary = geemap.shp_to_ee(shp_file)





# # everglades 
# shp_file = 'data/eden_boundary2.shp'
# small_boundary = geemap.shp_to_ee(shp_file)
# roi = small_boundary.geometry()
# roi_small = roi

In [4]:
# Import s1, s2, dynamic world data

d1 = '2020-01-01'
d2 = '2021-01-01'

# d1 = '2020-08-18'
# d2 = '2020-08-19'

# # get Dynamic World data
# dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
#     .filterBounds(s1_boundary)
#     .filterDate(d1, d2))    
# print('Number of images in dynamic world collection:', dw_collection.size().getInfo())
# # print('Bands:', dw_collection.first().bandNames().getInfo())


# # get Sentinel-2 data
# s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  
# print('Number of images in s2 collection:', s2_collection.size().getInfo())
# # print('Bands:', s2_collection.first().bandNames().getInfo())


# get Sentinel-1 data
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi_small))
print('Number of images in s1 collection:', s1_collection.size().getInfo())

Number of images in s1 collection: 31


In [5]:
# get dates of s1 images

acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['01-03-2020', '01-15-2020', '01-27-2020', '02-08-2020', '02-20-2020', '03-03-2020', '03-15-2020', '03-27-2020', '04-08-2020', '04-20-2020', '05-02-2020', '05-14-2020', '05-26-2020', '06-07-2020', '06-19-2020', '07-01-2020', '07-13-2020', '07-25-2020', '08-06-2020', '08-18-2020', '08-30-2020', '09-11-2020', '09-23-2020', '10-05-2020', '10-17-2020', '10-29-2020', '11-10-2020', '11-22-2020', '12-04-2020', '12-16-2020', '12-28-2020']


In [8]:
# get dates of dw images

acq_times = dw_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['01-01-2020', '01-01-2020', '01-01-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-03-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-06-2020', '01-08-2020', '01-08-2020', '01-08-2020', '01-08-2020', '01-08-2020', '01-08-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-11-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-16-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-18-2020', '01-21-2020', '01-21-2020', '01-2

In [7]:
# define function to implement speckle filter and get vv/vh ratio
def speckle(img):
    from speckle_filter import leesigma
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

s1_collection = s1_collection.map(speckle)

In [8]:
# print band names
print(s1_collection.first().bandNames().getInfo())

['VV', 'VH', 'angle', 'VV_VH', 'VV_filtered', 'VH_filtered', 'VV_VH_filtered']


In [9]:
# get bands of interest



# s1
s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered'])
# s1_img = s1_collection.select(['angle'])



vv_vis = {
    'min': 0,
    'max': 0.2,
    'palette': ['#000000','#ffffff']
}

vh_vis = {
    'min': 0,
    'max': 0.03,
    'palette': ['#000000','#ffffff']
}

vv_vh_vis = {
    'min': 3,
    'max': 30,
    'palette': ['#000000','#ffffff']
}



In [10]:
n = s1_collection.size().getInfo()
s1_collection_list = s1_collection.toList(n)
print(n)

1


In [11]:
ns = ee.Image(s1_collection_list.get(0)).select('VV_filtered').projection().nominalScale().getInfo()
print(ns)

10


In [12]:
type(s1_img)

ee.imagecollection.ImageCollection

In [13]:
s1_img = s1_collection.mean()

In [14]:
type(s1_img)

ee.image.Image

In [15]:
date_list

['03-03-2020']

In [16]:
# export s1 data in small region around nanda bet
task_id = []
n = 1
for i in range(0,n):
    img = ee.Image(s1_collection_list.get(i)).select(['VV_filtered','VH_filtered','VV_VH_filtered']).clip(small_boundary.geometry())
    #img = ee.Image(s1_collection_list.get(i)).select(['angle']).clip(small_boundary.geometry())
    #img = s1_img.select(['VV_filtered','VH_filtered','VV_VH_filtered']).clip(small_boundary.geometry())
    txt = 's1_nandabet_' + date_list[i] + '_20m'
    task = ee.batch.Export.image.toDrive(image=img,
                                        description=txt,
                                        scale=20,
                                        region=small_boundary.geometry(),
                                        fileNamePrefix=txt,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=1e9)
    task.start()
    task_id.append(task.id)
    print(date_list[i])
    


03-03-2020


In [19]:
task.list()

[<Task TQVBNT7GYYIRPAPG4CH4U6A5 EXPORT_IMAGE: s1_nandabet_03-03-2020_20m (COMPLETED)>,
 <Task OWIHZ7CGYJ4US6N2SN2XADKO EXPORT_IMAGE: dw_label_nandabet_20200301_20m (COMPLETED)>,
 <Task EHSTRGBKUSWYUIKY4B5QVZFX EXPORT_IMAGE: dw_bands_nandabet_20200301_20m (RUNNING)>,
 <Task BJGCCBABWUWE37IK5D7VORSB EXPORT_IMAGE: s2_nandabet_20200303_all_bands (COMPLETED)>,
 <Task BGORK4R5DYOO4W7PH2HGWYKW EXPORT_IMAGE: s1_nanda_bet_08-18-2020 (COMPLETED)>,
 <Task XO2LXBBZWYNR6W3ITQF5QMHD EXPORT_IMAGE: s1_everglades_12-06-2021 (COMPLETED)>,
 <Task OPNHMXYRWGANN2IFFQVHRK4Q EXPORT_IMAGE: s1_everglades_12-06-2021 (COMPLETED)>]

In [18]:
task.status()

{'state': 'COMPLETED',
 'description': 's1_nandabet_03-03-2020_20m',
 'creation_timestamp_ms': 1674762554464,
 'update_timestamp_ms': 1674762610196,
 'start_timestamp_ms': 1674762570689,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 32.714725494384766,
 'id': 'TQVBNT7GYYIRPAPG4CH4U6A5',
 'name': 'projects/earthengine-legacy/operations/TQVBNT7GYYIRPAPG4CH4U6A5'}

In [19]:
# define function to implement speckle filter and get vv/vh ratio
def speckle(img):
    from speckle_filter import leesigma
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

In [46]:
# get median backscatter by month for previous x years

yrs = ['2017','2018','2019']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

date_list = []
task_id = []

for m in range(len(months)):
    for y in range(len(yrs)):
        d1 = yrs[y] + '-' + months[m] + '-01'        
        if months[m] == '12':
            if yrs[y] == '2019':
                d2 = '2020-01-01'
            else:
                d2 = yrs[y+1] + '-01-01'
        else:
            d2 = yrs[y] + '-' + months[m+1] + '-01'
            

        if y == 0:
            # get Sentinel-1 data
            s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                             .filter(ee.Filter.eq('instrumentMode', 'IW'))
                             .filterMetadata('resolution_meters', 'equals', 10)
                             .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                             .filter(ee.Filter.date(d1,d2))
                             .filterBounds(roi_small))
            #print('Number of images in s1 collection:', s1_collection.size().getInfo())

            # apply speckle filter
            s1_collection = s1_collection.map(speckle)
            
        else:
            temp = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                             .filter(ee.Filter.eq('instrumentMode', 'IW'))
                             .filterMetadata('resolution_meters', 'equals', 10)
                             .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                             .filter(ee.Filter.date(d1,d2))
                             .filterBounds(roi_small))
            temp = temp.map(speckle)
            s1_collection = s1_collection.merge(temp)
            
    # get median by month and clip to small region
    s1_median = s1_collection.median().select(['VV_filtered','VH_filtered','VV_VH_filtered']).clip(small_boundary.geometry())


    # export s1 data in small region around nanda bet
    txt = 's1_median_' + yrs[y] + '-' + months[m]
    task = ee.batch.Export.image.toDrive(image=s1_median, description=txt, scale=10, region=small_boundary.geometry(),
                                        fileNamePrefix=txt, crs='EPSG:4326', fileFormat='GeoTIFF', maxPixels=1e9)
    task.start()
    task_id.append(task.id)


    # get dates of s1 images
    acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
    date_list.append([time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times])
        
print(date_list)

[['01-06-2017', '01-06-2017', '01-18-2017', '01-18-2017', '01-30-2017', '01-30-2017', '01-01-2018', '01-13-2018', '01-25-2018', '01-08-2019', '01-20-2019'], ['02-11-2017', '02-11-2017', '02-23-2017', '02-23-2017', '02-06-2018', '02-18-2018', '02-01-2019', '02-13-2019', '02-25-2019'], ['03-19-2017', '03-31-2017', '03-02-2018', '03-14-2018', '03-26-2018', '03-09-2019', '03-21-2019'], ['04-12-2017', '04-24-2017', '04-07-2018', '04-19-2018', '04-02-2019', '04-14-2019', '04-26-2019'], ['05-06-2017', '05-18-2017', '05-30-2017', '05-01-2018', '05-13-2018', '05-25-2018', '05-08-2019', '05-20-2019'], ['06-11-2017', '06-23-2017', '06-06-2018', '06-18-2018', '06-30-2018', '06-01-2019', '06-13-2019', '06-25-2019'], ['07-05-2017', '07-17-2017', '07-29-2017', '07-12-2018', '07-24-2018', '07-07-2019', '07-19-2019', '07-31-2019'], ['08-10-2017', '08-05-2018', '08-17-2018', '08-12-2019', '08-24-2019'], ['09-03-2017', '09-15-2017', '09-27-2017', '09-10-2018', '09-22-2018', '09-05-2019', '09-17-2019', '0

In [48]:
# save date_list

# open file in write mode
with open(r's1_median_dates.txt', 'w') as fp:
    for item in date_list:
        # write each item on a new line
        print(item)
        fp.write("%s\n" % item)
    print('Done')

['01-06-2017', '01-06-2017', '01-18-2017', '01-18-2017', '01-30-2017', '01-30-2017', '01-01-2018', '01-13-2018', '01-25-2018', '01-08-2019', '01-20-2019']
['02-11-2017', '02-11-2017', '02-23-2017', '02-23-2017', '02-06-2018', '02-18-2018', '02-01-2019', '02-13-2019', '02-25-2019']
['03-19-2017', '03-31-2017', '03-02-2018', '03-14-2018', '03-26-2018', '03-09-2019', '03-21-2019']
['04-12-2017', '04-24-2017', '04-07-2018', '04-19-2018', '04-02-2019', '04-14-2019', '04-26-2019']
['05-06-2017', '05-18-2017', '05-30-2017', '05-01-2018', '05-13-2018', '05-25-2018', '05-08-2019', '05-20-2019']
['06-11-2017', '06-23-2017', '06-06-2018', '06-18-2018', '06-30-2018', '06-01-2019', '06-13-2019', '06-25-2019']
['07-05-2017', '07-17-2017', '07-29-2017', '07-12-2018', '07-24-2018', '07-07-2019', '07-19-2019', '07-31-2019']
['08-10-2017', '08-05-2018', '08-17-2018', '08-12-2019', '08-24-2019']
['09-03-2017', '09-15-2017', '09-27-2017', '09-10-2018', '09-22-2018', '09-05-2019', '09-17-2019', '09-29-2019